In [ ]:
# **YOLO v5 모델로 히잡, 마스크 객체 감지 실습**

*   고영민 (https://psleon.tistory.com/)



In [ ]:
# install YOLO v5
!git clone https://github.com/ultralytics/yolov5.git

# 1. **데이터셋 준비**

In [6]:
!pip install -r ./yolov5/requirements.txt

                                              0.0/7.6 MB ? eta -:--:--
     -                                        0.4/7.6 MB 11.2 MB/s eta 0:00:01
     ----                                     0.9/7.6 MB 11.7 MB/s eta 0:00:01
     -------                                  1.5/7.6 MB 11.6 MB/s eta 0:00:01
     ----------                               2.0/7.6 MB 11.6 MB/s eta 0:00:01
     -------------                            2.5/7.6 MB 11.6 MB/s eta 0:00:01
     ----------------                         3.1/7.6 MB 11.7 MB/s eta 0:00:01
     -------------------                      3.6/7.6 MB 11.7 MB/s eta 0:00:01
     ----------------------                   4.2/7.6 MB 11.7 MB/s eta 0:00:01
     ------------------------                 4.7/7.6 MB 11.7 MB/s eta 0:00:01
     ---------------------------              5.3/7.6 MB 11.7 MB/s eta 0:00:01
     ------------------------------           5.8/7.6 MB 11.6 MB/s eta 0:00:01
     ---------------------------------        6.4/7.6 MB 11.

In [ ]:
!dir

In [ ]:
!type hijab.yaml

In [1]:
import yaml

with open('hijab.yaml', 'r') as f:
  data = yaml.full_load(f)

print(data)

data['train'] = '../hijab/images/train/'
data['val'] = '../hijab/images/val/'

with open('hijab.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['burqz', 'chador', 'mask', 'niqab'], 'nc': 4, 'train': '../hijab/images/train/', 'val': '../hijab/images/val/'}
{'names': ['burqz', 'chador', 'mask', 'niqab'], 'nc': 4, 'train': '../hijab/images/train/', 'val': '../hijab/images/val/'}


In [3]:
import torch
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print(device)
print(torch.cuda.get_device_name(0)) #-> 그래픽카드 이름 추출

cuda:0
NVIDIA GeForce RTX 4080


In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# **2.모델 학습**
* data : data yaml 파일 경로 (데이터셋 정보가 적힌 yaml 파일)
* weights : Pre-Trained 모델 파일 경로 (pt 형식 파일), 아무런 값을 적지 않으면 (‘’) 랜덤한 weight 값으로 초기화 및 학습 진행
* epochs : epoch 수
* batch : batch_size
* cfg : yolo v5 아키텍쳐 yaml 파일 경로 yolo v5는 s, m, l, x의 4가지 버전이 있음
  * s가 가장 가벼운 모델 x가 가장 무거운 모델 당연히 s가 성능이 제일 낮지만 FPS가 가장 높고, x가 성능이 제일 높지만 FPS는 가장 낮다.

In [7]:
# epochs 50
!python ./yolov5/train.py --img 640 --batch 16 --epochs 50 --data hijab.yaml --cfg ./yolov5/models/yolov5s.yaml --weights yolov5s.pt --name hijab_yolov5s_results

train: weights=yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=hijab.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=hijab_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 5 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5  v7.0-187-g0004c74 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1,

In [ ]:
# epochs 100
from tqdm import tqdm
!python ./yolov5/train.py --img 640 --batch 16 --epochs 100 --data hijab.yaml --cfg ./yolov5/models/yolov5s.yaml --weights yolov5s.pt --name hijab_yolov5s_results2

학습이 끝난 모델은 ./yolov5/runs/ 내에 train.py ~ --name으로 지정해준 이름의 디렉터리에 저장된다.
본 연습에서는 ./yolov5/runs/train/hijab_yolov5s_results/ 디렉터리에 저장되며 해당 경로에 results.png를 통해 학습 결과를 볼 수 있다. 

### 텐서보드로 학습 결과 보기

In [3]:
%load_ext tensorboard
%tensorboard --logdir ./yolov5/runs/

Reusing TensorBoard on port 6006 (pid 3496), started 5 days, 23:21:56 ago. (Use '!kill 3496' to kill it.)

# 3. **모델 검증**

학습한 모델 가중치는 /content/yolov5/runs/train/hijab_yolov5s_results/weights/best.pt에 저장되며 이를 이용해 모델을 검증한다.

In [ ]:
!python ./yolov5/val.py --data ./hijab.yaml --weights ./yolov5/runs/train/hijab_yolov5s_results/weights/best.pt

검증 결과는 /runs/val/exp/ 에 저장된다.

# 4. **모델 예측**
* source : 테스트 이미지 (혹은 폴더) 경로
* weights : 학습이 완료된 weight 파일 경로 (pt 형식)
* conf : conf_threshold 값 (0 ~ 1 사이의 값)으로

class score가 설정한 값을 넘겨야, 바운딩 박스를 그린다.<br>
runs/detect/exp 경로에 결과가 저장된다.<br>
예측을 반복하면 runs/detect 경로에 exp1, 2, 3… 같은 형태로 폴더가 생성되면서 결과가 기록된다.

In [ ]:
from glob import glob
test_img_list = glob('./hijab/images/test/**/*.jpeg', recursive=True)
test_img_list2 = glob('./hijab/images/test/**/*.jpg', recursive=True)
print("length:", len(test_img_list), len(test_img_list2))
print(test_img_list)
print(test_img_list2)

test_dataset = test_img_list + test_img_list2
print(len(test_dataset))

In [ ]:
import os
from IPython.display import Image

In [ ]:
val_img_path = test_dataset[30]
!python ./yolov5/detect.py --weights ./yolov5/runs/train/hijab_yolov5s_results/weights/best.pt --source "{val_img_path}"
Image(os.path.join('./yolov5/runs/detect/exp/', os.path.basename(val_img_path)))

In [ ]:
val_img_path = test_dataset[14]
!python ./yolov5/detect.py --weights ./yolov5/runs/train/hijab_yolov5s_results/weights/best.pt --source "{val_img_path}"
Image(os.path.join('./yolov5/runs/detect/exp2/', os.path.basename(val_img_path)))